In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Sangmoo, I live in South Korea", "Wow that is so cold!")

get_history()

{'history': [HumanMessage(content="Hi I'm Sangmoo, I live in South Korea"),
  AIMessage(content='Wow that is so cold!')]}

In [2]:
add_message("South Korea is so pretty", "I wish I could go!!!")
get_history()

{'history': [HumanMessage(content="Hi I'm Sangmoo, I live in South Korea"),
  AIMessage(content='Wow that is so cold!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!')]}

In [3]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
get_history()

{'history': [HumanMessage(content="Hi I'm Sangmoo, I live in South Korea"),
  AIMessage(content='Wow that is so cold!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}